In [ ]:
!pip install transformers
!pip install torch
!pip install sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.6 MB/s eta 0:00:00
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


In [ ]:
def generate_response(input_text):
    new_user_input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response

def extract_info(input_text):
    # assume input_text is of the format "My salary is <salary> and I have <passive_income> passive income."
    info = input_text.lower().split()
    income = int(info[3])
    passive_income = 0 if 'no' in info else int(info[7])
    return income, passive_income

def calculate_tax(income, passive_income):
    taxable_income = income - passive_income
    if taxable_income <= 250000:
        return 0
    elif taxable_income <= 500000:
        return (taxable_income - 250000) * 0.05
    elif taxable_income <= 750000:
        return 12500 + (taxable_income - 500000) * 0.1
    elif taxable_income <= 1000000:
        return 37500 + (taxable_income - 750000) * 0.15
    elif taxable_income <= 1250000:
        return 75000 + (taxable_income - 1000000) * 0.20
    elif taxable_income <= 1500000:
        return 125000 + (taxable_income - 1250000) * 0.25
    else:
        return 187500 + (taxable_income - 1500000) * 0.30

chat_history_ids = None

for step in range(5):
    input_text = input(">> User:")


    if "calculate my income tax" in input_text.lower():
        print("Bot: Sure! What is your salary and passive income?")
        input_text = input(">> User:")
        income, passive_income = extract_info(input_text)
        tax = calculate_tax(income, passive_income)
        print(f"Bot: Your tax amount is: {tax}")
    elif "thank you" in input_text.lower():
        print("Bot: You're welcome!")
        break
    else:
        response = generate_response(input_text)
        print("DialoGPT: {}".format(response))


>> User:can you help me calculate my income tax
Bot: Sure! What is your salary and passive income?
>> User:My salary is 700000 and I have no passive income.
Bot: Your tax amount is: 32500.0
>> User:Thank you
Bot: You're welcome!
